# minion instance
{
	"mid": 1,
	"uid": 1,
	"level": 60,
	"tot_xp": 150000,
	"xp_to_next_level": 6000,
    "tot_stats_pt": 136,
    "assigned_stats_pt": 136,
    "free_stats_pt": 0,
	"base_stats": [35, 20, 15, 23, 10, 17, 26],  # str, end, dex, obd, con, int, wis; given by stats points
	#"stats_ceiling":[100, 100, 100, 100, 100, 100, 100],
	"cur_stats": [44, 25, 19, 23, 10, 17, 26],  # base_stats * wellness_factor * luck
	"base_substats": [150, 130, 0.15, -0.20],  # atk, def, crit, oby; f(base_stats)
	"cur_substats": [150, 130, 0.15, -0.20], # f(current_stats)
	"base_attributes_capacity": [2000, 500, 500],  # hit points, ability points, energy points; f(base_stats)
	"cur_attributes_capacity": [1900, 550, 550], # hit points, ability points, energy points; f(current_stats)
    "base_attributes": [1900, 490, 485], # keep track of depletion and replenishment, clipped by base_attributes_ceiling
    "cur_attribues": [1900, 490, 485], # base_attributes * luck, clipped by current_attributes_ceiling? problem: attibute ceiling is fluctuating
	"states": [0,0,0],
	"base_transition_matrix": [[0.5, 0.3, 0.2],
            [0.25, 0.5, 0.25],
            [0.2, 0.3, 0.5]],
	"cur_transition_matrix": [[0.5, 0.3, 0.2],
            [0.25, 0.5, 0.25],
            [0.2, 0.3, 0.5]], # f(base_transition_matrix, alpha)
	"wellness_goal": [1,1,1,1,1] # fast food vs grocery, gym vs movie, ...
    "wellness_factor": 1.25, # f(wellness_goal, transaction or user activities)
	"luck": [-0.05, 0.2, -0.1, -0.15], # stats, attributes, skills, abilities
	"cp": 2456, # f(current_substats, current_attributes)
	"skills": [{"sid": 1, "name": "fast_move_1", "type": "fmove", "cd": 0.5, "energy": 20, "power": 100}, 
               {"sid": 2, "name": "charge_move_1", "type": "cmove", "cd": 10, "energy": -200, "power": 1500}]
	#"abilities": []
}


# skill instance
{
    "sid": 1,
    "name": "",
    "type": "fast_move_1",
    "cd": 0.5,
    "energy": 20,  # in traditional RPG, energy/MP increases during battle by making fmove/overtime. = energy points in our game?
    "power": 100
}

{
    "sid": 2,
    "name": "",
    "type": "cmove",
    "cd": 10,
    "energy": -200,
    "power": 1500
}

# buff and debuff?

# coin instance
{
    "type": 2, # 0: soul, 1: mind, 2: body
    "strength": 0.2,
    "xp": 50,
    "attributes": [20, 3, 5], # replenish hit points, ability points, energy points
    "wellness_category": [1, None, None, None, None], 
    "weight": 1.3 # given by f(minion's wellness_goal, wellness_category). computed at feeding
}


# ability instance

# draft functions

In [1]:
import numpy as np
import random

In [53]:
minion_1 = {
	"mid": 1,
	"uid": 1,
	"level": 60,
	"tot_xp": 150000,
	"xp_to_next_level": 6000,
    "tot_stats_pt": 136,
    "assigned_stats_pt": 136,
    "free_stats_pt": 0,
	"base_stats": [35, 20, 15, 23, 10, 17, 26],  # str, end, dex, obd, con, int, wis; given by stats points
	#"stats_ceiling":[100, 100, 100, 100, 100, 100, 100],
	"cur_stats": [],  # base_stats * wellness_factor * luck
	"base_substats": [],  # atk, def, crit, succ; f(base_stats)
	"cur_substats": [], # f(current_stats)
	"base_attributes_capacity": [],  # hit points, ability points, energy points; f(base_stats)
	"cur_attributes_capacity": [], # hit points, ability points, energy points; f(current_stats)
    "base_attributes": [1900, 490, 485], # keep track of depletion and replenishment, clipped by base_attributes_ceiling
    "cur_attributes": [], # base_attributes * luck, clipped by current_attributes_ceiling? problem: attibute ceiling is fluctuating
	"states": [0,0,0],
    "stats_coeff": [[5, 3, 0.005, 0.015, 85, 50, 70], #coefficents
                    [60, 30, 0.20, 0.50, 500, 500, 500]], #initial substats and attributes at lvl 0
	"base_transition_matrix": [[0.5, 0.3, 0.2],
            [0.25, 0.5, 0.25],
            [0.2, 0.3, 0.5]],
	"cur_transition_matrix": [[0.5, 0.3, 0.2],
            [0.25, 0.5, 0.25],
            [0.2, 0.3, 0.5]], # f(base_transition_matrix, alpha)
	"wellness_goal": [1,1,1,1,1], # fast food vs grocery, gym vs movie, ...
    "wellness_factor": 1.25, # f(wellness_goal, transaction or user activities)
	"luck": [], # stats, attributes, skills, abilities
	"cp": 0, # f(cur_substats, cur_attributes)
	"skills": {0:{"sid": 1, "name": "fast_move_1", "type": "fmove", "cd": 0.5, "cost": 5, "power": 20}, 
               1:{"sid": 2, "name": "charge_move_1", "type": "cmove", "cd": 10, "cost": -80, "power": 800}},
	"abilities": {}
}

In [ ]:
def compute_stats_pt(minion):
    
    minion['assigned_stats_pt'] = sum(np.array(minion['base_stats']))
    minion['tot_stats_pt'] = 
    minion['free_stats_pt'] = minion['tot_stats_pt'] - minion['assigned_stats_pt']
    return minion

In [48]:
# strength, endurance, dexteiry, obedience, constitution, intelligence, wisdom
# attack, defense, critical, success, hit points, ability points, energy points 
# https://na.leagueoflegends.com/en/game-info/champions/Ashe/
base_stats_coeff_1 = [[5, 3, 0.005, 0.015, 85, 50, 70], #coefficents
                      [60, 30, 0.20, 0.50, 500, 500, 500]] #initial substats and attributes at lvl 0

def generate_stats_coeff(minion): #randomize initial substats and attributes by 10%
    stats_coeff = np.array(minion['stats_coeff'])
    rand_mat = np.array([random.uniform(0.9,1.1) for _ in range(len(stats_coeff[1]))])
    stats_coeff[1] = np.multiply(stats_coeff[1], rand_mat)
    stats_coeff[1][0:2] = np.floor(stats_coeff[1][0:2]) + 1
    stats_coeff[1][4:7] = np.floor(stats_coeff[1][4:7]) + 1
    minion['stats_coeff'] = stats_coeff
    return minion


In [44]:
def generate_luck(minion, p = [0.2, 0.65, 0.15], magnitude = 0.3):
    # p: luck distribution, [bad, neutural, good]
    n_stats = len(minion['base_stats'])
    n_attributes = len(minion['base_attributes'])
    n_skills = len(minion['skills'])
    n_abilities = len(minion['abilities'])
    
    stats_luck = np.array([np.random.choice(1+np.arange(-1,2)*magnitude, p=p) for _ in range(n_stats)])
    attributes_luck = np.array([np.random.choice(1+np.arange(-1,2)*magnitude, p=p) for _ in range(n_attributes)])
    skills_luck = np.array([np.random.choice(1-np.arange(-1,2)*magnitude, p=p) for _ in range(n_skills)])
    abilities_luck = np.array([np.random.choice(1-np.arange(-1,2)*magnitude, p=p) for _ in range(n_abilities)])

    luck = np.array([stats_luck, attributes_luck, skills_luck, abilities_luck])
    
    minion['luck'] = luck
    return minion

In [50]:
def compute_current_stats(minion): # f(base_stats, base_attributes, luck, wellness_factor)
    
    def clip_by_capacity(attr, attr_capacity):
        for i in range(len(attr)):
            attr[i] = min(attr[i], attr_capacity[i])
        return attr
    
    base_stats = np.array(minion['base_stats'])
    base_attributes = np.array(minion['base_attributes'])
    luck = np.array(minion['luck'])
    wellness_factor = minion['wellness_factor']
    stats_coeff = np.array(minion['stats_coeff'])
    
    # base_capacity = f(base_stats)
    base_capacity = np.floor(np.multiply(base_stats, stats_coeff[0]) + stats_coeff[1]) + 1
    base_substats = base_capacity[0:4]
    base_attributes_capacity = base_capacity[4:7]
    base_attributes = clip_by_capacity(base_attributes, base_attributes_capacity)
    
    # cur_stats = f(base_stats, luck, wellness_factor)
    cur_stats = np.floor((np.multiply(base_stats, luck[0]) * wellness_factor)) + 1
    
    # cur_capacity = f(cur_stats)
    cur_capacity = np.floor(np.multiply(cur_stats, stats_coeff[0]) + stats_coeff[1]) + 1
    cur_substats = cur_capacity[0:4]
    cur_attributes_capacity = cur_capacity[4:7]
    
    # cur_attributes = f(base_attributes, luck)
    cur_attributes = clip_by_capacity(np.floor(np.multiply(base_attributes, luck[1])) + 1, cur_attributes_capacity)
    
    # tbd: cp = f(cur_substats, cur_attributes, skill's DPS, ...)
    #cp = 
    
    
    # update
    minion['base_substats'] = base_substats
    minion['base_attributes_capacity'] = base_attributes_capacity
    minion['base_attributes'] = base_attributes
    minion['cur_stats'] = cur_stats
    minion['cur_substats'] = cur_substats
    minion['cur_attributes_capacity'] = cur_attributes_capacity
    minion['cur_attributes'] = cur_attributes
    #minion['cp'] = cp

    return minion

In [54]:
minion_1 = generate_stats_coeff(minion_1)
minion_1 = generate_luck(minion_1)
minion_1 = compute_current_stats(minion_1)

In [55]:
minion_1

{'mid': 1,
 'uid': 1,
 'level': 60,
 'tot_xp': 150000,
 'xp_to_next_level': 6000,
 'tot_stats_pt': 136,
 'assigned_stats_pt': 136,
 'free_stats_pt': 0,
 'base_stats': [35, 20, 15, 23, 10, 17, 26],
 'cur_stats': array([57., 26., 14., 29.,  9., 22., 23.]),
 'base_substats': array([239.,  92.,   1.,   1.]),
 'cur_substats': array([349., 110.,   1.,   1.]),
 'base_attributes_capacity': array([1357., 1318., 2364.]),
 'cur_attributes_capacity': array([1272., 1568., 2154.]),
 'base_attributes': array([1357,  490,  485]),
 'cur_attributes': array([1272.,  344.,  486.]),
 'states': [0, 0, 0],
 'stats_coeff': array([[5.00000000e+00, 3.00000000e+00, 5.00000000e-03, 1.50000000e-02,
         8.50000000e+01, 5.00000000e+01, 7.00000000e+01],
        [6.30000000e+01, 3.10000000e+01, 2.03257630e-01, 4.98200923e-01,
         5.06000000e+02, 4.67000000e+02, 5.43000000e+02]]),
 'base_transition_matrix': [[0.5, 0.3, 0.2],
  [0.25, 0.5, 0.25],
  [0.2, 0.3, 0.5]],
 'cur_transition_matrix': [[0.5, 0.3, 0.2],


In [ ]:
def give_coin(minion, coin):
    
    return minion

In [ ]:
def assign_stats_pt(minion, stats_type, num_pt=1):
    #stats_type: 0:str, 1:end, 2:dex, 3:obd, 4:con, 5:int, 6:wis
    
    return minion

# battle simulator

In [ ]:
def compute_attack_damage(attacker, defender, skill_id):
    
    def compute_cost(skill, skill_luck):
        if skill['type'] == 'fmove': # fmove builds up mp?
            cost = skill['cost'] * (2-skill_luck)
        if skill['type'] == 'cmove':
            cost = skill['cost'] * skill_luck
        return cost
    
    def is_critical(critical_rate): # yes: 2 (critical hit creates 2x damage); no: 1
        return np.random.choice(np.arange(1,3), p=[1-critical_rate, critical_rate])
    
    def is_success(success_rate): # yes: 1; no: 0
        return np.random.choice(np.arange(0,2), p=[1-success_rate, success_rate])
    
    #a_substats = attacker['cur_substats']
    #d_substats = defender['cur_substats']
    #a_attr = attacker['cur_attribues']
    #d_attr = defender['cur_attribues'] 
    
    a_atk = attacker['cur_substats'][0]
    a_critical_rate = attacker['cur_substats'][2]
    a_success_rate = attacker['cur_substats'][3]
    d_def = defender['cur_substats'][1]
    
    skill = attacker['skills'][skill_id]
    skill_luck = attacker['luck'][2][skill_id]
    cost = compute_cost(skill, skill_luck)
    power = skill['power']
    
    a_ep = attacker['cur_attribues'][2]
    d_hp = defender['cur_attribues'][0]
    
    # tbd: damage function
    damage = (np.floor(a_atk/d_def * is_critical(a_critical_rate) * power) + 1) * is_success(a_success_rate)
    attacker['cur_attribues'][2] = a_ep - cost
    defender['cur_attribues'][0] = d_hp - damage
    
    # attacker: mp
    # defender: hp
    return attacker, defenser